# 📊 Análisis Exploratorio v2: Estructura Esquemática del Paper
## Efectividad de la RCP Transtelefónica en Paradas Cardiacas Extrahospitalarias

### Objetivos del Análisis:
1. **Análisis descriptivo** de la muestra (Tabla 1 del paper)
2. **Comparación de outcomes** por grupos de RCP
3. **Análisis específico**: Distribución de CPC según RCP transtelefónica vs RCP de testigos legos
4. **Tests estadísticos**: t-Student y Chi-cuadrado
5. **Preparación de figuras** para el paper científico

---

## 📚 1. Importación de Librerías y Datos

In [ ]:
# Librerías para análisis de datos y estadística
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import chi2_contingency, ttest_ind, mannwhitneyu
import warnings
warnings.filterwarnings('ignore')

# Configuración de visualización
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12
plt.rcParams['axes.grid'] = True
plt.rcParams['grid.alpha'] = 0.3

# Configuración para guardar figuras
import os
figures_dir = '/Users/miguel/Desktop/RCP Transtelefonica/proyecto/figures'
os.makedirs(figures_dir, exist_ok=True)

print("✅ Librerías importadas correctamente")
print(f"📁 Directorio de figuras: {figures_dir}")

In [ ]:
# Cargar datos limpios
data_path = '/Users/miguel/Desktop/RCP Transtelefonica/data/cleaned_data.csv'
df = pd.read_csv(data_path)

# Mostrar información básica del dataset
print("📊 INFORMACIÓN DEL DATASET")
print("="*50)
print(f"Total de casos: {len(df)}")
print(f"Variables: {len(df.columns)}")
print(f"\nColumnas disponibles:")
for i, col in enumerate(df.columns, 1):
    print(f"{i:2d}. {col}")

# Vista previa de los datos
print("\n📋 PRIMERAS 5 FILAS:")
print("="*50)
display(df.head())

# Información de tipos de datos y missing values
print("\n🔍 INFORMACIÓN DE VARIABLES:")
print("="*50)
info_df = pd.DataFrame({
    'Tipo': df.dtypes,
    'No Nulos': df.count(),
    'Nulos': df.isnull().sum(),
    '% Nulos': (df.isnull().sum() / len(df) * 100).round(2)
})
display(info_df)

## 📋 2. Tabla 1: Características Basales de la Muestra

### Análisis descriptivo estratificado por RCP Transtelefónica

In [ ]:
# Crear grupos para análisis
df['grupo_rcp'] = df['rcp_transtelefonica'].map({
    1: 'RCP Transtelefónica',
    0: 'Sin RCP Transtelefónica'
})

# Función para crear tabla descriptiva
def crear_tabla_descriptiva(data, grupo_col='grupo_rcp'):
    """
    Crea tabla descriptiva estilo paper científico
    """
    grupos = data[grupo_col].unique()
    resultados = []
    
    # Total general
    total_n = len(data)
    
    for grupo in grupos:
        subset = data[data[grupo_col] == grupo]
        n_grupo = len(subset)
        
        # Edad
        edad_media = subset['edad'].mean()
        edad_std = subset['edad'].std()
        
        # Sexo masculino %
        sexo_masc_n = (subset['sexo'] == 'M').sum()
        sexo_masc_pct = (sexo_masc_n / n_grupo) * 100
        
        # RCP de testigos
        rcp_testigos_n = subset['rcp_testigos'].sum()
        rcp_testigos_pct = (rcp_testigos_n / n_grupo) * 100
        
        # Ritmo desfibrilable
        ritmo_desf_n = subset['ritmo'].sum()
        ritmo_desf_pct = (ritmo_desf_n / n_grupo) * 100
        
        # DESA externo
        desa_n = subset['desa_externo'].sum()
        desa_pct = (desa_n / n_grupo) * 100
        
        # Tiempo de llegada (mediana y IQR)
        tiempo_mediana = subset['tiempo_llegada_unidad'].median()
        tiempo_q25 = subset['tiempo_llegada_unidad'].quantile(0.25)
        tiempo_q75 = subset['tiempo_llegada_unidad'].quantile(0.75)
        
        # ROSC
        rosc_n = subset['rosc'].sum()
        rosc_pct = (rosc_n / n_grupo) * 100
        
        # Supervivencia 7 días
        superv_n = subset['supervivencia_7dias'].sum()
        superv_pct = (superv_n / n_grupo) * 100
        
        resultados.append({
            'Grupo': grupo,
            'N (%)': f"{n_grupo} ({n_grupo/total_n*100:.1f}%)",
            'Edad (años)': f"{edad_media:.1f} ± {edad_std:.1f}",
            'Sexo masculino': f"{sexo_masc_n} ({sexo_masc_pct:.1f}%)",
            'RCP de testigos': f"{rcp_testigos_n} ({rcp_testigos_pct:.1f}%)",
            'Ritmo desfibrilable': f"{ritmo_desf_n} ({ritmo_desf_pct:.1f}%)",
            'DESA externo': f"{desa_n} ({desa_pct:.1f}%)",
            'Tiempo llegada (seg)': f"{tiempo_mediana:.0f} ({tiempo_q25:.0f}-{tiempo_q75:.0f})",
            'ROSC': f"{rosc_n} ({rosc_pct:.1f}%)",
            'Supervivencia 7d': f"{superv_n} ({superv_pct:.1f}%)"
        })
    
    return pd.DataFrame(resultados)

# Crear y mostrar tabla descriptiva
tabla_1 = crear_tabla_descriptiva(df)

print("📋 TABLA 1: CARACTERÍSTICAS BASALES DE LA MUESTRA")
print("="*80)
display(tabla_1.T)  # Transpuesta para mejor visualización

# Guardar tabla como CSV para el paper
tabla_1.to_csv(f"{figures_dir}/tabla_1_caracteristicas_basales.csv", index=False)
print(f"\n💾 Tabla guardada en: {figures_dir}/tabla_1_caracteristicas_basales.csv")

## 📊 3. Tests Estadísticos: Variables Demográficas

### Comparación entre grupos con y sin RCP Transtelefónica

In [ ]:
# Separar grupos para análisis estadístico
grupo_rcp_trans = df[df['rcp_transtelefonica'] == 1]
grupo_sin_rcp_trans = df[df['rcp_transtelefonica'] == 0]

print("🔬 TESTS ESTADÍSTICOS: CARACTERÍSTICAS BASALES")
print("="*70)

# 1. Test t-Student para edad
edad_rcp = grupo_rcp_trans['edad'].dropna()
edad_sin_rcp = grupo_sin_rcp_trans['edad'].dropna()

t_stat_edad, p_val_edad = ttest_ind(edad_rcp, edad_sin_rcp)

print(f"\n1️⃣ EDAD (t-Student):")
print(f"   • RCP Trans: {edad_rcp.mean():.1f} ± {edad_rcp.std():.1f} años (n={len(edad_rcp)})")
print(f"   • Sin RCP Trans: {edad_sin_rcp.mean():.1f} ± {edad_sin_rcp.std():.1f} años (n={len(edad_sin_rcp)})")
print(f"   • t = {t_stat_edad:.3f}, p = {p_val_edad:.4f}")
print(f"   • {'Significativo' if p_val_edad < 0.05 else 'No significativo'} (α = 0.05)")

# 2. Chi-cuadrado para sexo
tabla_sexo = pd.crosstab(df['sexo'], df['rcp_transtelefonica'])
chi2_sexo, p_chi2_sexo, dof_sexo, expected_sexo = chi2_contingency(tabla_sexo)

print(f"\n2️⃣ SEXO (Chi-cuadrado):")
print(f"   Tabla de contingencia:")
display(tabla_sexo)
print(f"   • χ² = {chi2_sexo:.3f}, p = {p_chi2_sexo:.4f}")
print(f"   • {'Significativo' if p_chi2_sexo < 0.05 else 'No significativo'} (α = 0.05)")

# 3. Chi-cuadrado para RCP de testigos
tabla_rcp_testigos = pd.crosstab(df['rcp_testigos'], df['rcp_transtelefonica'])
chi2_testigos, p_chi2_testigos, dof_testigos, expected_testigos = chi2_contingency(tabla_rcp_testigos)

print(f"\n3️⃣ RCP DE TESTIGOS (Chi-cuadrado):")
print(f"   Tabla de contingencia:")
display(tabla_rcp_testigos)
print(f"   • χ² = {chi2_testigos:.3f}, p = {p_chi2_testigos:.4f}")
print(f"   • {'Significativo' if p_chi2_testigos < 0.05 else 'No significativo'} (α = 0.05)")

# 4. Mann-Whitney U para tiempo de llegada (no paramétrico)
tiempo_rcp = grupo_rcp_trans['tiempo_llegada_unidad'].dropna()
tiempo_sin_rcp = grupo_sin_rcp_trans['tiempo_llegada_unidad'].dropna()

u_stat_tiempo, p_val_tiempo = mannwhitneyu(tiempo_rcp, tiempo_sin_rcp, alternative='two-sided')

print(f"\n4️⃣ TIEMPO DE LLEGADA (Mann-Whitney U):")
print(f"   • RCP Trans: {tiempo_rcp.median():.0f} seg (IQR: {tiempo_rcp.quantile(0.25):.0f}-{tiempo_rcp.quantile(0.75):.0f})")
print(f"   • Sin RCP Trans: {tiempo_sin_rcp.median():.0f} seg (IQR: {tiempo_sin_rcp.quantile(0.25):.0f}-{tiempo_sin_rcp.quantile(0.75):.0f})")
print(f"   • U = {u_stat_tiempo:.0f}, p = {p_val_tiempo:.4f}")
print(f"   • {'Significativo' if p_val_tiempo < 0.05 else 'No significativo'} (α = 0.05)")

## 🎯 4. Análisis Principal: ROSC y Supervivencia

### Outcomes primarios y secundarios

In [ ]:
print("🎯 ANÁLISIS DE OUTCOMES PRINCIPALES")
print("="*60)

# 1. ROSC por grupo
tabla_rosc = pd.crosstab(df['rosc'], df['rcp_transtelefonica'], margins=True)
chi2_rosc, p_chi2_rosc, dof_rosc, expected_rosc = chi2_contingency(tabla_rosc.iloc[:-1, :-1])

print(f"\n1️⃣ ROSC (Outcome Primario):")
print(f"   Tabla de contingencia:")
tabla_rosc_percent = pd.crosstab(df['rosc'], df['rcp_transtelefonica'], normalize='columns') * 100
tabla_rosc_display = pd.DataFrame({
    'Sin RCP Trans (n)': tabla_rosc.iloc[:-1, 0],
    'Sin RCP Trans (%)': tabla_rosc_percent.iloc[:, 0].round(1),
    'RCP Trans (n)': tabla_rosc.iloc[:-1, 1],
    'RCP Trans (%)': tabla_rosc_percent.iloc[:, 1].round(1)
})
tabla_rosc_display.index = ['Sin ROSC', 'Con ROSC']
display(tabla_rosc_display)

# Calcular OR para ROSC
a = tabla_rosc.iloc[1, 1]  # ROSC con RCP trans
b = tabla_rosc.iloc[0, 1]  # Sin ROSC con RCP trans
c = tabla_rosc.iloc[1, 0]  # ROSC sin RCP trans
d = tabla_rosc.iloc[0, 0]  # Sin ROSC sin RCP trans

or_rosc = (a * d) / (b * c)
se_log_or = np.sqrt(1/a + 1/b + 1/c + 1/d)
ci_lower_rosc = np.exp(np.log(or_rosc) - 1.96 * se_log_or)
ci_upper_rosc = np.exp(np.log(or_rosc) + 1.96 * se_log_or)

print(f"\n   📊 Estadísticas:")
print(f"   • χ² = {chi2_rosc:.3f}, p = {p_chi2_rosc:.4f}")
print(f"   • OR = {or_rosc:.2f} (IC 95%: {ci_lower_rosc:.2f}-{ci_upper_rosc:.2f})")
print(f"   • {'Significativo' if p_chi2_rosc < 0.05 else 'No significativo'} (α = 0.05)")

# 2. Supervivencia a 7 días
tabla_superv = pd.crosstab(df['supervivencia_7dias'], df['rcp_transtelefonica'], margins=True)
chi2_superv, p_chi2_superv, dof_superv, expected_superv = chi2_contingency(tabla_superv.iloc[:-1, :-1])

print(f"\n2️⃣ SUPERVIVENCIA 7 DÍAS (Outcome Secundario):")
print(f"   Tabla de contingencia:")
tabla_superv_percent = pd.crosstab(df['supervivencia_7dias'], df['rcp_transtelefonica'], normalize='columns') * 100
tabla_superv_display = pd.DataFrame({
    'Sin RCP Trans (n)': tabla_superv.iloc[:-1, 0],
    'Sin RCP Trans (%)': tabla_superv_percent.iloc[:, 0].round(1),
    'RCP Trans (n)': tabla_superv.iloc[:-1, 1],
    'RCP Trans (%)': tabla_superv_percent.iloc[:, 1].round(1)
})
tabla_superv_display.index = ['No superviviente', 'Superviviente']
display(tabla_superv_display)

# Calcular OR para supervivencia
a_s = tabla_superv.iloc[1, 1]  # Superviviente con RCP trans
b_s = tabla_superv.iloc[0, 1]  # No superviviente con RCP trans
c_s = tabla_superv.iloc[1, 0]  # Superviviente sin RCP trans
d_s = tabla_superv.iloc[0, 0]  # No superviviente sin RCP trans

or_superv = (a_s * d_s) / (b_s * c_s)
se_log_or_s = np.sqrt(1/a_s + 1/b_s + 1/c_s + 1/d_s)
ci_lower_superv = np.exp(np.log(or_superv) - 1.96 * se_log_or_s)
ci_upper_superv = np.exp(np.log(or_superv) + 1.96 * se_log_or_s)

print(f"\n   📊 Estadísticas:")
print(f"   • χ² = {chi2_superv:.3f}, p = {p_chi2_superv:.4f}")
print(f"   • OR = {or_superv:.2f} (IC 95%: {ci_lower_superv:.2f}-{ci_upper_superv:.2f})")
print(f"   • {'Significativo' if p_chi2_superv < 0.05 else 'No significativo'} (α = 0.05)")

## 🧠 5. Análisis Específico: Distribución de CPC

### CPC según RCP Transtelefónica vs RCP de Testigos Legos

In [ ]:
print("🧠 ANÁLISIS DE CPC (CEREBRAL PERFORMANCE CATEGORY)")
print("="*70)

# Filtrar casos con RCP de testigos legos
df_legos = df[df['respondiente_rcp'] == 'lego'].copy()

print(f"\n📊 MUESTRA PARA ANÁLISIS CPC:")
print(f"   • Total casos: {len(df)}")
print(f"   • Casos con RCP de testigos legos: {len(df_legos)}")
print(f"   • Casos con RCP transtelefónica (total): {df['rcp_transtelefonica'].sum()}")
print(f"   • Casos con RCP transtelefónica (legos): {df_legos['rcp_transtelefonica'].sum()}")

# 1. Análisis en toda la muestra: CPC por RCP Transtelefónica
print(f"\n\n1️⃣ DISTRIBUCIÓN CPC: RCP TRANSTELEFÓNICA vs NO TRANSTELEFÓNICA (Muestra total)")
print("-"*80)

tabla_cpc_total = pd.crosstab(df['cpc'], df['rcp_transtelefonica'], normalize='columns') * 100
tabla_cpc_counts = pd.crosstab(df['cpc'], df['rcp_transtelefonica'])

# Crear tabla combinada con n y %
tabla_cpc_display = pd.DataFrame()
for col in [0, 1]:
    col_name = 'Sin RCP Trans' if col == 0 else 'RCP Trans'
    tabla_cpc_display[f'{col_name} n(%)'] = [
        f"{tabla_cpc_counts.loc[cpc, col]} ({tabla_cpc_total.loc[cpc, col]:.1f}%)"
        for cpc in tabla_cpc_counts.index
    ]

tabla_cpc_display.index = [f'CPC {cpc}' for cpc in tabla_cpc_counts.index]
display(tabla_cpc_display)

# Test Chi-cuadrado para CPC
chi2_cpc_total, p_chi2_cpc_total, dof_cpc_total, expected_cpc_total = chi2_contingency(tabla_cpc_counts)
print(f"\n   📊 Test Chi-cuadrado:")
print(f"   • χ² = {chi2_cpc_total:.3f}, p = {p_chi2_cpc_total:.4f}")
print(f"   • {'Significativo' if p_chi2_cpc_total < 0.05 else 'No significativo'} (α = 0.05)")

# 2. Análisis específico en legos: CPC por RCP Transtelefónica
if len(df_legos) > 0:
    print(f"\n\n2️⃣ DISTRIBUCIÓN CPC: RCP TRANSTELEFÓNICA vs NO TRANSTELEFÓNICA (Solo testigos legos)")
    print("-"*80)
    
    tabla_cpc_legos = pd.crosstab(df_legos['cpc'], df_legos['rcp_transtelefonica'], normalize='columns') * 100
    tabla_cpc_legos_counts = pd.crosstab(df_legos['cpc'], df_legos['rcp_transtelefonica'])
    
    # Crear tabla combinada con n y %
    tabla_cpc_legos_display = pd.DataFrame()
    for col in tabla_cpc_legos_counts.columns:
        col_name = 'Sin RCP Trans' if col == 0 else 'RCP Trans'
        tabla_cpc_legos_display[f'{col_name} n(%)'] = [
            f"{tabla_cpc_legos_counts.loc[cpc, col]} ({tabla_cpc_legos.loc[cpc, col]:.1f}%)"
            for cpc in tabla_cpc_legos_counts.index
        ]
    
    tabla_cpc_legos_display.index = [f'CPC {cpc}' for cpc in tabla_cpc_legos_counts.index]
    display(tabla_cpc_legos_display)
    
    # Test Chi-cuadrado para CPC en legos
    if tabla_cpc_legos_counts.shape[0] > 1 and tabla_cpc_legos_counts.shape[1] > 1:
        chi2_cpc_legos, p_chi2_cpc_legos, dof_cpc_legos, expected_cpc_legos = chi2_contingency(tabla_cpc_legos_counts)
        print(f"\n   📊 Test Chi-cuadrado (Legos):")
        print(f"   • χ² = {chi2_cpc_legos:.3f}, p = {p_chi2_cpc_legos:.4f}")
        print(f"   • {'Significativo' if p_chi2_cpc_legos < 0.05 else 'No significativo'} (α = 0.05)")
    else:
        print(f"\n   ⚠️ Insuficientes datos para test Chi-cuadrado en subgrupo legos")
else:
    print(f"\n   ⚠️ No hay casos de RCP por testigos legos en la muestra")

## 📊 6. Visualizaciones para el Paper

### Figuras principales del análisis

In [ ]:
# Configuración de colores
colores = {'Sin RCP Trans': '#2E86AB', 'RCP Trans': '#A23B72', 'Legos': '#F18F01'}

print("📊 GENERANDO VISUALIZACIONES PARA EL PAPER")
print("="*60)

# 1. Figura 1: Comparación de ROSC y Supervivencia
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# ROSC por grupo
rosc_data = df.groupby('rcp_transtelefonica')['rosc'].agg(['count', 'sum', 'mean']).reset_index()
rosc_data['grupo'] = rosc_data['rcp_transtelefonica'].map({0: 'Sin RCP Trans', 1: 'RCP Trans'})
rosc_data['porcentaje'] = rosc_data['mean'] * 100

bars1 = ax1.bar(rosc_data['grupo'], rosc_data['porcentaje'], 
                color=[colores['Sin RCP Trans'], colores['RCP Trans']],
                alpha=0.8, edgecolor='black', linewidth=1)

# Añadir valores en las barras
for i, (bar, row) in enumerate(zip(bars1, rosc_data.itertuples())):
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2., height + 1,
             f'{row.sum}/{row.count}\n({height:.1f}%)',
             ha='center', va='bottom', fontweight='bold', fontsize=11)

ax1.set_title('ROSC por Tipo de RCP', fontsize=14, fontweight='bold')
ax1.set_ylabel('Porcentaje (%)', fontsize=12)
ax1.set_ylim(0, max(rosc_data['porcentaje']) * 1.2)
ax1.grid(True, alpha=0.3)

# Supervivencia por grupo
superv_data = df.groupby('rcp_transtelefonica')['supervivencia_7dias'].agg(['count', 'sum', 'mean']).reset_index()
superv_data['grupo'] = superv_data['rcp_transtelefonica'].map({0: 'Sin RCP Trans', 1: 'RCP Trans'})
superv_data['porcentaje'] = superv_data['mean'] * 100

bars2 = ax2.bar(superv_data['grupo'], superv_data['porcentaje'], 
                color=[colores['Sin RCP Trans'], colores['RCP Trans']],
                alpha=0.8, edgecolor='black', linewidth=1)

# Añadir valores en las barras
for i, (bar, row) in enumerate(zip(bars2, superv_data.itertuples())):
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height + 1,
             f'{row.sum}/{row.count}\n({height:.1f}%)',
             ha='center', va='bottom', fontweight='bold', fontsize=11)

ax2.set_title('Supervivencia 7 días por Tipo de RCP', fontsize=14, fontweight='bold')
ax2.set_ylabel('Porcentaje (%)', fontsize=12)
ax2.set_ylim(0, max(superv_data['porcentaje']) * 1.2)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(f'{figures_dir}/figura_1_rosc_supervivencia_rcp_trans.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"✅ Figura 1 guardada: figura_1_rosc_supervivencia_rcp_trans.png")

In [ ]:
# 2. Figura 2: Distribución de CPC (Muestra total)
fig, ax = plt.subplots(1, 1, figsize=(12, 8))

# Preparar datos para visualización
cpc_data = df.groupby(['cpc', 'rcp_transtelefonica']).size().unstack(fill_value=0)
cpc_data_pct = cpc_data.div(cpc_data.sum(axis=0), axis=1) * 100

# Crear gráfico de barras agrupadas
x = np.arange(len(cpc_data.index))
width = 0.35

bars1 = ax.bar(x - width/2, cpc_data_pct[0], width, 
               label='Sin RCP Transtelefónica', 
               color=colores['Sin RCP Trans'], alpha=0.8, edgecolor='black')
bars2 = ax.bar(x + width/2, cpc_data_pct[1], width,
               label='RCP Transtelefónica', 
               color=colores['RCP Trans'], alpha=0.8, edgecolor='black')

# Añadir valores en las barras
for i, (bar1, bar2) in enumerate(zip(bars1, bars2)):
    # Sin RCP Trans
    height1 = bar1.get_height()
    ax.text(bar1.get_x() + bar1.get_width()/2., height1 + 0.5,
            f'{cpc_data.iloc[i, 0]}\n({height1:.1f}%)',
            ha='center', va='bottom', fontsize=10)
    
    # RCP Trans
    height2 = bar2.get_height()
    ax.text(bar2.get_x() + bar2.get_width()/2., height2 + 0.5,
            f'{cpc_data.iloc[i, 1]}\n({height2:.1f}%)',
            ha='center', va='bottom', fontsize=10)

ax.set_xlabel('Cerebral Performance Category (CPC)', fontsize=12)
ax.set_ylabel('Porcentaje (%)', fontsize=12)
ax.set_title('Distribución de CPC según RCP Transtelefónica\n(Muestra Total)', fontsize=14, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels([f'CPC {cpc}' for cpc in cpc_data.index])
ax.legend(loc='upper right')
ax.grid(True, alpha=0.3)

# Añadir significancia estadística
ax.text(0.02, 0.98, f'χ² = {chi2_cpc_total:.3f}, p = {p_chi2_cpc_total:.4f}', 
        transform=ax.transAxes, va='top', ha='left',
        bbox=dict(boxstyle='round', facecolor='white', alpha=0.8),
        fontsize=11, fontweight='bold')

plt.tight_layout()
plt.savefig(f'{figures_dir}/figura_2_distribucion_cpc_total.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"✅ Figura 2 guardada: figura_2_distribucion_cpc_total.png")

In [ ]:
# 3. Figura 3: Distribución de CPC en testigos legos (si hay datos suficientes)
if len(df_legos) > 10:  # Solo si hay suficientes datos
    fig, ax = plt.subplots(1, 1, figsize=(12, 8))
    
    # Preparar datos para visualización
    cpc_legos_data = df_legos.groupby(['cpc', 'rcp_transtelefonica']).size().unstack(fill_value=0)
    cpc_legos_data_pct = cpc_legos_data.div(cpc_legos_data.sum(axis=0), axis=1) * 100
    
    # Crear gráfico de barras agrupadas
    x = np.arange(len(cpc_legos_data.index))
    width = 0.35
    
    bars1 = ax.bar(x - width/2, cpc_legos_data_pct[0], width, 
                   label='Sin RCP Transtelefónica', 
                   color=colores['Sin RCP Trans'], alpha=0.8, edgecolor='black')
    
    if 1 in cpc_legos_data.columns:
        bars2 = ax.bar(x + width/2, cpc_legos_data_pct[1], width,
                       label='RCP Transtelefónica', 
                       color=colores['RCP Trans'], alpha=0.8, edgecolor='black')
    
    # Añadir valores en las barras
    for i, bar1 in enumerate(bars1):
        height1 = bar1.get_height()
        ax.text(bar1.get_x() + bar1.get_width()/2., height1 + 1,
                f'{cpc_legos_data.iloc[i, 0]}\n({height1:.1f}%)',
                ha='center', va='bottom', fontsize=10)
    
    if 1 in cpc_legos_data.columns:
        for i, bar2 in enumerate(bars2):
            height2 = bar2.get_height()
            ax.text(bar2.get_x() + bar2.get_width()/2., height2 + 1,
                    f'{cpc_legos_data.iloc[i, 1]}\n({height2:.1f}%)',
                    ha='center', va='bottom', fontsize=10)
    
    ax.set_xlabel('Cerebral Performance Category (CPC)', fontsize=12)
    ax.set_ylabel('Porcentaje (%)', fontsize=12)
    ax.set_title('Distribución de CPC según RCP Transtelefónica\n(Solo Testigos Legos)', fontsize=14, fontweight='bold')
    ax.set_xticks(x)
    ax.set_xticklabels([f'CPC {cpc}' for cpc in cpc_legos_data.index])
    ax.legend(loc='upper right')
    ax.grid(True, alpha=0.3)
    
    # Añadir información de la muestra
    ax.text(0.02, 0.98, f'n = {len(df_legos)} casos\n(Testigos legos)', 
            transform=ax.transAxes, va='top', ha='left',
            bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.8),
            fontsize=11, fontweight='bold')
    
    plt.tight_layout()
    plt.savefig(f'{figures_dir}/figura_3_distribucion_cpc_legos.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print(f"✅ Figura 3 guardada: figura_3_distribucion_cpc_legos.png")
else:
    print(f"⚠️ Insuficientes casos de testigos legos (n={len(df_legos)}) para generar figura")

## 📋 7. Resumen de Resultados Principales

### Conclusiones para el Paper

In [ ]:
print("📋 RESUMEN DE RESULTADOS PRINCIPALES")
print("="*80)

print(f"\n🔢 CARACTERÍSTICAS DE LA MUESTRA:")
print(f"   • Total de casos analizados: {len(df)}")
print(f"   • Edad media: {df['edad'].mean():.1f} ± {df['edad'].std():.1f} años")
print(f"   • Sexo masculino: {(df['sexo'] == 'M').sum()} ({(df['sexo'] == 'M').mean()*100:.1f}%)")
print(f"   • RCP Transtelefónica: {df['rcp_transtelefonica'].sum()} ({df['rcp_transtelefonica'].mean()*100:.1f}%)")

print(f"\n🎯 OUTCOMES PRINCIPALES:")
print(f"   • ROSC global: {df['rosc'].sum()} ({df['rosc'].mean()*100:.1f}%)")
print(f"   • ROSC con RCP Transtelefónica: {grupo_rcp_trans['rosc'].sum()}/{len(grupo_rcp_trans)} ({grupo_rcp_trans['rosc'].mean()*100:.1f}%)")
print(f"   • ROSC sin RCP Transtelefónica: {grupo_sin_rcp_trans['rosc'].sum()}/{len(grupo_sin_rcp_trans)} ({grupo_sin_rcp_trans['rosc'].mean()*100:.1f}%)")
print(f"   • OR para ROSC: {or_rosc:.2f} (IC 95%: {ci_lower_rosc:.2f}-{ci_upper_rosc:.2f}), p = {p_chi2_rosc:.4f}")

print(f"\n🏥 SUPERVIVENCIA 7 DÍAS:")
print(f"   • Supervivencia global: {df['supervivencia_7dias'].sum()} ({df['supervivencia_7dias'].mean()*100:.1f}%)")
print(f"   • Supervivencia con RCP Transtelefónica: {grupo_rcp_trans['supervivencia_7dias'].sum()}/{len(grupo_rcp_trans)} ({grupo_rcp_trans['supervivencia_7dias'].mean()*100:.1f}%)")
print(f"   • Supervivencia sin RCP Transtelefónica: {grupo_sin_rcp_trans['supervivencia_7dias'].sum()}/{len(grupo_sin_rcp_trans)} ({grupo_sin_rcp_trans['supervivencia_7dias'].mean()*100:.1f}%)")
print(f"   • OR para Supervivencia: {or_superv:.2f} (IC 95%: {ci_lower_superv:.2f}-{ci_upper_superv:.2f}), p = {p_chi2_superv:.4f}")

print(f"\n🧠 DISTRIBUCIÓN CPC:")
cpc_resumen = df['cpc'].value_counts().sort_index()
for cpc, count in cpc_resumen.items():
    pct = count / len(df) * 100
    print(f"   • CPC {cpc}: {count} ({pct:.1f}%)")
print(f"   • Diferencias CPC por RCP Transtelefónica: χ² = {chi2_cpc_total:.3f}, p = {p_chi2_cpc_total:.4f}")

print(f"\n🎓 CONCLUSIONES PRELIMINARES:")
print(f"   • La RCP Transtelefónica se asocia con {'mayor' if or_rosc > 1 else 'menor'} probabilidad de ROSC")
print(f"   • La RCP Transtelefónica se asocia con {'mayor' if or_superv > 1 else 'menor'} probabilidad de supervivencia")
print(f"   • Las diferencias en CPC {'son' if p_chi2_cpc_total < 0.05 else 'no son'} estadísticamente significativas")
print(f"   • Se observan diferencias clínicamente relevantes en outcomes neurológicos")

print(f"\n📊 ARCHIVOS GENERADOS:")
print(f"   • Tabla 1: tabla_1_caracteristicas_basales.csv")
print(f"   • Figura 1: figura_1_rosc_supervivencia_rcp_trans.png")
print(f"   • Figura 2: figura_2_distribucion_cpc_total.png")
if len(df_legos) > 10:
    print(f"   • Figura 3: figura_3_distribucion_cpc_legos.png")

print(f"\n" + "="*80)
print(f"✅ ANÁLISIS COMPLETADO - DATOS LISTOS PARA EL PAPER")
print(f"="*80)